In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mipkit
from radiomics.featureextractor import RadiomicsFeatureExtractor


# https://github.com/AIM-Harvard/pyradiomics/issues/490
import logging
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

# ... or set level for specific class
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)


This section contains the definitions of the various features that can be extracted using PyRadiomics. They are subdivided into the following classes:

- First Order Statistics (19 features)
- Shape-based (3D) (16 features)
- Shape-based (2D) (10 features)
- Gray Level Cooccurence Matrix (24 features)
- Gray Level Run Length Matrix (16 features)
- Gray Level Size Zone Matrix (16 features)
- Neighbouring Gray Tone Difference Matrix (5 features)
- Gray Level Dependence Matrix (14 features)

Source: https://pyradiomics.readthedocs.io/en/latest/features.html

In [3]:
# Instantiate the extractor
texture_extractor = RadiomicsFeatureExtractor(verbose=False)
texture_extractor.disableAllFeatures()
_text_feat = {ckey: [] for ckey in texture_extractor.featureClassNames}
texture_extractor.enableFeaturesByName(**_text_feat)

print('Extraction parameters:\n\t', texture_extractor.settings)
print('Enabled filters:\n\t', texture_extractor.enabledImagetypes)
print('Enabled features:\n\t', texture_extractor.enabledFeatures)


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'verbose': False}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': [], 'shape2D': []}


In [4]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np  # for manipulating 3d images
import pandas as pd  # for reading and writing tables
import os

import mipkit

%matplotlib inline
plt.rcParams["figure.figsize"] = (4, 4)
plt.rcParams["figure.dpi"] = 125
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid': False})


In [5]:
def calc_radiomics(ct_mask, seg_mask):
    return texture_extractor.execute(imageFilepath=mipkit.medical.cvt_array_to_ITKImage(ct_mask),
                                     maskFilepath=mipkit.medical.cvt_array_to_ITKImage((seg_mask).astype(np.uint8)))


In [6]:
# /mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_Mask_n=2687
SEG_PATH = '/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_Mask_n=2687/LC00026.nii'
CT_PATH = '/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_CT_n=2687/LC00026.nii'
seg_mask = mipkit.medical.load_medical_image_file(SEG_PATH)
ct_image = mipkit.medical.load_medical_image_file(CT_PATH)

In [7]:
plt.imshow(seg_mask.sum(axis=0))

In [8]:
%time
radiomic_feature_results = texture_extractor.execute(imageFilepath=mipkit.medical.cvt_array_to_ITKImage(ct_image),
                                                     maskFilepath=mipkit.medical.cvt_array_to_ITKImage((seg_mask).astype(np.uint8)))


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 2.62 µs


In [9]:
# pd.DataFrame([results])['diagnostics_Image-original_Spacing']
# pd.DataFrame([results])['original_ngtdm_Strength']

In [10]:
def get_manual_selected_radiomic_features(radiomic_feature_path):
    with open(radiomic_feature_path, 'r') as f:
        lines = f.readlines()
    return [feature.strip() for feature in lines]


radiomic_feature = get_manual_selected_radiomic_features(
    'manual_selected_radiomic_feature.txt')

radiomic_feature_results = pd.DataFrame([radiomic_feature_results])[radiomic_feature]

In [11]:
radiomic_feature_results

diagnostics_Image-original_Size  diagnostics_Image-original_Mean  \
0                 (256, 256, 148)                        39.213625   

   diagnostics_Image-original_Minimum  diagnostics_Image-original_Maximum  \
0                                 0.0                               255.0   

  diagnostics_Mask-original_Size diagnostics_Mask-original_BoundingBox  \
0                (256, 256, 148)              (54, 77, 84, 50, 53, 12)   

   diagnostics_Mask-original_VoxelNum  diagnostics_Mask-original_VolumeNum  \
0                               17543                                    1   

         diagnostics_Mask-original_CenterOfMassIndex  \
0  (78.07136749700736, 103.30439491535085, 89.796...   

              diagnostics_Mask-original_CenterOfMass  ...  \
0  (78.07136749700736, 103.30439491535085, 89.796...  ...   

   original_glszm_SmallAreaHighGrayLevelEmphasis  \
0                             3.3720436971693126   

   original_glszm_SmallAreaLowGrayLevelEmphasis  original_glszm_ZoneEntropy  \
0                            0.2142588538254373           3.546439344671011   

   original_glszm_ZonePercentage original_glszm_ZoneVariance  \
0           0.001140055862737274           4993021.527500001   

  original_ngtdm_Busyness original_ngtdm_Coarseness original_ngtdm_Complexity  \
0       93.49116741901166     0.0008416313573779905          2.53912638570691   

  original_ngtdm_Contrast  original_ngtdm_Strength  
0     0.01587617898801243     0.010394365112801389  

[1 rows x 117 columns]

In [12]:
radiomic_feature_results.describe()

diagnostics_Image-original_Mean  diagnostics_Image-original_Minimum  \
count                         1.000000                                 1.0   
mean                         39.213625                                 0.0   
std                                NaN                                 NaN   
min                          39.213625                                 0.0   
25%                          39.213625                                 0.0   
50%                          39.213625                                 0.0   
75%                          39.213625                                 0.0   
max                          39.213625                                 0.0   

       diagnostics_Image-original_Maximum  diagnostics_Mask-original_VoxelNum  \
count                                 1.0                                 1.0   
mean                                255.0                             17543.0   
std                                   NaN                                 NaN   
min                                 255.0                             17543.0   
25%                                 255.0                             17543.0   
50%                                 255.0                             17543.0   
75%                                 255.0                             17543.0   
max                                 255.0                             17543.0   

       diagnostics_Mask-original_VolumeNum  original_shape_Elongation  \
count                                  1.0                   1.000000   
mean                                   1.0                   0.955353   
std                                    NaN                        NaN   
min                                    1.0                   0.955353   
25%                                    1.0                   0.955353   
50%                                    1.0                   0.955353   
75%                                    1.0                   0.955353   
max                                    1.0                   0.955353   

       original_shape_Flatness  original_shape_LeastAxisLength  \
count                 1.000000                        1.000000   
mean                  0.225313                       10.914117   
std                        NaN                             NaN   
min                   0.225313                       10.914117   
25%                   0.225313                       10.914117   
50%                   0.225313                       10.914117   
75%                   0.225313                       10.914117   
max                   0.225313                       10.914117   

       original_shape_MajorAxisLength  original_shape_MinorAxisLength  \
count                        1.000000                        1.000000   
mean                        48.439786                       46.277086   
std                               NaN                             NaN   
min                         48.439786                       46.277086   
25%                         48.439786                       46.277086   
50%                         48.439786                       46.277086   
75%                         48.439786                       46.277086   
max                         48.439786                       46.277086   

       original_shape_VoxelVolume  
count                         1.0  
mean                      17543.0  
std                           NaN  
min                       17543.0  
25%                       17543.0  
50%                       17543.0  
75%                       17543.0  
max                       17543.0

In [13]:
import pandas as pd

# Load Clinical Information
PATH = '/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/LC_NSCLC_CLINICAL_n=2687_20210202.xlsx'
df_clinical = pd.read_excel(PATH)

# Remove unnecessary columns
df_clinical = df_clinical[['PatientID', 'Survival.time', 'Deadstatus.event']]
df_clinical

PatientID  Survival.time  Deadstatus.event
0      LC00009             74                 1
1      LC00012            330                 1
2      LC00013            411                 1
3      LC00021             29                 1
4      LC00022            548                 1
...        ...            ...               ...
2682   LC10091            842                 0
2683   LC10093            585                 0
2684   LC10094              8                 9
2685   LC10096            798                 0
2686   LC10097            526                 9

[2687 rows x 3 columns]

In [14]:
from queue import Queue
# from tqdm import tqdm
from tqdm.auto import tqdm

def init_extractor():
    radiomic_extractor = RadiomicsFeatureExtractor(verbose=False)
    radiomic_extractor.disableAllFeatures()
    _text_feat = {ckey: [] for ckey in radiomic_extractor.featureClassNames}
    radiomic_extractor.enableFeaturesByName(**_text_feat)
    return radiomic_extractor

In [16]:
# def extract_radiomic_multithread(inputs):
#     shared_extractor_queue, inp = inputs
#     ct_mask_path, seg_mask_path = inp

#     while not shared_extractor_queue.empty():
#         extractor = shared_extractor_queue.get()
#         results = calc_radiomics(extractor, ct_mask_path, seg_mask_path)
#         shared_extractor_queue.put(extractor)
#         return results


# NUM_WORKERS = 2

# radiomic_extractor_queue = Queue()
# [radiomic_extractor_queue.put(init_extractor()) for _ in range(NUM_WORKERS)]

# print(radiomic_extractor_queue.qsize())


# inputs = []        
# for patient_id in tqdm(df_clinical.PatientID):
#     if patient_id != 'LC02380':
#         seg_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_Mask_n=2687/{patient_id}.nii'
#         ct_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_CT_n=2687/{patient_id}.nii'
#         inputs.append([radiomic_extractor_queue, (ct_path, seg_path)])


# results = mipkit.multiprocessing.pool_worker(inputs=inputs,
#                                              target=extract_radiomic_multithread,
#                                              num_worker=NUM_WORKERS, 
#                                              use_thread=True)



In [17]:

# results = mipkit.multiprocessing.pool_worker(inputs=inputs,
#                                              target=extract_radiomic_multithread,
#                                              num_worker=NUM_WORKERS, 
#                                              use_thread=True)


In [18]:
def calc_radiomics(extractor, ct_mask_path, seg_mask_path):
    seg_mask = mipkit.medical.load_medical_image_file(seg_mask_path)
    ct_mask = mipkit.medical.load_medical_image_file(ct_mask_path)

    if len(np.unique(seg_mask)) >= 2:
        return extractor.execute(imageFilepath=mipkit.medical.cvt_array_to_ITKImage(ct_mask),
                                 maskFilepath=mipkit.medical.cvt_array_to_ITKImage((seg_mask).astype(np.uint8)))
    else:
        return None

def extract_radiomic_multiprocess(inputs):
    i, inputs = inputs
    extractor = init_extractor()
    results = []
    print('Process ', i)
    pbar = tqdm(total=len(inputs), position=i+1, leave=False, desc='Process {}'.format(i))
    for patient_id, ct_mask_path, seg_mask_path in inputs:
        
        result = calc_radiomics(extractor, ct_mask_path, seg_mask_path)
        if result is not None:
            result['patient_id'] = patient_id
        results.append(result)
        
        pbar.update(1)
    return results

inputs = []        
for patient_id in tqdm(df_clinical.PatientID):
    if patient_id != 'LC02380':
        seg_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_Mask_n=2687/{patient_id}.nii'
        ct_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_CT_n=2687/{patient_id}.nii'
        inputs.append([patient_id, ct_path, seg_path])

splited_inputs = mipkit.partition_a_list(inputs, num_parts=6)

splited_inputs = [[i, splited_input] for i, splited_input in enumerate(splited_inputs)]
len(splited_inputs)

results = mipkit.multiprocessing.pool_worker(inputs=splited_inputs,
                                             target=extract_radiomic_multiprocess,
                                             num_worker=6)


Multi-Process:   0%|          | 0/6 [00:00<?, ?it/s]

Process  Process 0Process Process  
 Process  Process 12 3 

4
5



Multi-Process: 100%|██████████| 6/6 [13:27<00:00, 134.54s/it]  


In [21]:
# from tqdm import tqdm
# # Some warnings should be noted
# # Warning: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
# list_valid_patient_id = []
# for patient_id in tqdm(df_clinical.PatientID):
#     if patient_id != 'LC02380':
#         seg_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_Mask_n=2687/{patient_id}.nii'
#         ct_path = f'/mnt/mssd1T/LungCancer/LC_NSCLC_Cleaned_n=2687_20210202/PreCrop_LC_NSCLC_CT_n=2687/{patient_id}.nii'
#         seg_mask = mipkit.medical.load_medical_image_file(seg_path)
#         ct_image = mipkit.medical.load_medical_image_file(ct_path)

#         if len(np.unique(seg_mask)) >= 2:
#             list_valid_patient_id.append(patient_id)


In [23]:
# len(list_valid_patient_id)

In [24]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [25]:
final_results = flatten(results)

In [26]:
len(final_results)

2686

In [27]:
# Remove Invalid Data
final_results = [res for res in final_results if res is not None]

In [28]:
len(final_results)

2679

In [30]:
import numpy as np

np.save('final_results_radiomic_features.npy', final_results)

In [31]:
# mipkit.partition_a_list(list(range(0, 13)))

In [32]:
final_results_df = pd.DataFrame(final_results)
final_results_df = final_results_df.drop(columns=['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
                                                  'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
                                                  'diagnostics_Versions_Python', 'diagnostics_Configuration_EnabledImageTypes',
                                                  'diagnostics_Configuration_Settings', 'diagnostics_Image-original_Hash', 
                                                  'diagnostics_Image-original_Spacing','diagnostics_Image-original_Size', 
                                                  'diagnostics_Mask-original_Hash', 'diagnostics_Image-original_Dimensionality',
                                                  'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Image-original_Maximum',
                                                  'diagnostics_Image-original_Minimum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex',
                                                  'diagnostics_Mask-original_CenterOfMass', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox'])


In [33]:
# def is_number(inp):
#     if isinstance(inp, int):
#         return True
#     elif isinstance(inp, float):
#         return True
#     else:
#         False

In [34]:
# for key, val in final_results_df.iloc[0].items():
#     if not is_number(val):
#         print(key, val)

In [35]:
final_results_df.to_csv('radiomic.csv', index=False)

In [ ]:
# Reconstruct and save a DataFrame 
# all_df = pd.DataFrame({'h5_path': list(data_root.glob('*.h5'))})
# all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
# all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
# all_df['scan_id'] = all_df['file_id'].map(lambda x: int(x.split('_')[-1]))
# all_df.head(5) # show the first 5 lines

# all_df['radiomics'] = all_df['h5_path'].map(lambda c_filename: calc_radiomics(read_scan(c_filename)))

# full_df = pd.DataFrame([dict(**c_row.pop('radiomics'), **c_row) for _, c_row in all_df.iterrows()])
# print(full_df.shape, 'data prepared')
# first_cols = all_df.columns[:-1].tolist()
# full_df = full_df[first_cols + [c_col for c_col in full_df.columns 
#                                 if c_col not in first_cols]]
# # export the whole table
# full_df.to_csv('all_radiomics_table.csv', index=False)
# full_df.sample(3)

In [ ]:
# Focusing on Interesting Radiomics

# leave out anything that doesn't start with original (just junk from the input)
# also remove shape since it is not very informative
# value_feature_names = [c_col for c_col in full_df.columns if (c_col.startswith('original') and '_shape_' not in c_col)]
# print(np.random.choice(value_feature_names, 3), 'of', len(value_feature_names))

# make a cleaner vefirst_colson
# clean_df = full_df[first_cols + value_feature_names].copy()
# clean_df.columns = first_cols + [
#     ' '.join(c_col.split('original_')[-1].split('_')) 
#     for c_col in value_feature_names
# ]
# clean_col_names = clean_df.columns[len(first_cols):]
# fix some of the artifacts from the radiomics tool
# obj_cols = clean_df[clean_col_names].select_dtypes(['object']).columns.tolist()
# for c_col in obj_cols:
#     clean_df[c_col] = clean_df[c_col].map(lambda x: float(x))
# clean_df.to_csv('clean_radiomics_table.csv', index=False)
# clean_df.sample(3)

In [ ]:
# Show the distribution of different variables
# fig, m_axs = plt.subplots(12, 8, figsize=(60, 70))
# for c_ax, c_var in zip(m_axs.flatten(), clean_col_names):
#     c_ax.hist(clean_df[c_var].values, bins=10)
#     c_ax.set_title('\n'.join(c_var.split(' ')))
# fig.savefig('all_metrics.png')